In [1]:
# Upload CSV file
from google.colab import files
uploaded = files.upload()


TypeError: 'NoneType' object is not subscriptable

In [ ]:
!pip install openpyxl --quiet

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re


# Display plots inline
%matplotlib inline


In [2]:
# Load the uploaded file
file_path = "Mutilation Data.xlsx"

# Read the first sheet (or specify sheet name with sheet_name="Sheet1")
df = pd.read_excel(file_path)

# Preview the data
df.head()


NameError: name 'pd' is not defined

In [ ]:
# Replace dashes in DEFECT_LOCATION column with "UNSPECIFIED"
df['DEFECT_LOCATION'] = df['DEFECT_LOCATION'].replace('--', 'UNSPECIFIED')

In [3]:
# Count defects per day
defect_counts = df.groupby(['weekday', 'day']).size().reset_index(name='defect_count')

# Clean up trailing spaces in weekday column
defect_counts['weekday'] = defect_counts['weekday'].str.strip()

# Define the correct order for days of the week (Monday to Sunday)
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Create box plot with specified order
plt.figure(figsize=(10, 6))
sns.boxplot(data=defect_counts, x='weekday', y='defect_count', order=day_order)
plt.title('Distribution of Daily Defects by Day of Week')
plt.xlabel('Day of Week')
plt.ylabel('Number of Defects')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

NameError: name 'df' is not defined

In [4]:
# Convert CAPTURED_TIME to datetime
df['captured_time_dt'] = pd.to_datetime(df['CAPTURED_TIME'])

# Extract hour and day of week
df['hour'] = df['captured_time_dt'].dt.hour
df['day_of_week'] = df['captured_time_dt'].dt.day_name()

# Plotting
plt.figure(figsize=(12, 5))

# 1. Defects by Hour of Day
plt.subplot(1, 2, 1)
hourly_defects = df['hour'].value_counts().sort_index()
hourly_defects.plot(kind='bar', color='steelblue')
plt.title('Defects by Hour of Day')
plt.xlabel('Hour')
plt.ylabel('Number of Defects')
plt.xticks(rotation=0)

# 2. Defects by Day of Week
plt.subplot(1, 2, 2)
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_defects = df['day_of_week'].value_counts().reindex(day_order)
daily_defects.plot(kind='bar', color='coral')
plt.title('Defects by Day of Week')
plt.xlabel('Day')
plt.ylabel('Number of Defects')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


NameError: name 'pd' is not defined

In [5]:


# Your existing part extraction function
def extract_main_part(defect_place):
    if pd.isna(defect_place):
        return 'Unknown'

    # Convert to lowercase for matching
    text = str(defect_place).lower()

    # Define part keywords in order of priority (more specific first)
    part_patterns = {
        'Door': ['door'],
        'Panel': ['panel'],
        'Seat': ['seat'],
        'Pillar': ['pillar', 'b-pillar'],
        'Fender': ['fender'],
        'Sidewall': ['sidewall'],
        'Cover': ['cover'],
        'Cockpit': ['cockpit'],
        'Radio': ['radio'],
        'Mirror': ['mirror'],
        'Window': ['window'],
        'Hood': ['hood'],
        'Trunk': ['trunk'],
        'Bumper': ['bumper'],
        'Tire': ['tire'],
        'Wheel': ['wheel'],
        'Light': ['light', 'headlight', 'taillight'],
        'Windshield': ['windshield'],
        'Roof': ['roof'],
        'Floor': ['floor'],
        'Dashboard': ['dashboard', 'dash'],
        'Console': ['console'],
        'Handle': ['handle'],
        'Lock': ['lock'],
        'Hinge': ['hinge'],
        'Trim': ['trim'],
        'Molding': ['molding'],
        'Grill': ['grill', 'grille']
    }

    # Check for each part pattern
    for part_name, keywords in part_patterns.items():
        for keyword in keywords:
            if keyword in text:
                return part_name

    # If no match found, try to extract the first meaningful word
    # Remove common prefixes and suffixes
    clean_text = re.sub(r'\b(code1|code2|code3|check|inspect|verify|defect)\b', '', text)

    # Extract first meaningful word (at least 3 characters)
    words = re.findall(r'\b[a-zA-Z]{3,}\b', clean_text)
    if words:
        return words[0].title()

    return 'Other'

# Function to extract defect types (you may need to adjust column name)
def extract_defect_type(defect_description):
    if pd.isna(defect_description):
        return 'Unknown'

    text = str(defect_description).lower()

    # Define defect type patterns - adjust these based on your actual defect descriptions
    defect_patterns = {
        'Scratch': ['scratch', 'scratched', 'scuff'],
        'Dent': ['dent', 'dented', 'ding'],
        'Crack': ['crack', 'cracked', 'split'],
        'Damage': ['damage', 'damaged', 'broken'],
        'Wear': ['wear', 'worn', 'faded'],
        'Missing': ['missing', 'absent', 'not present'],
        'Loose': ['loose', 'not tight', 'wobbly'],
        'Misaligned': ['misaligned', 'crooked', 'off-center'],
        'Stain': ['stain', 'stained', 'dirty'],
        'Tear': ['tear', 'torn', 'rip']
    }

    # Check for each defect pattern
    for defect_name, keywords in defect_patterns.items():
        for keyword in keywords:
            if keyword in text:
                return defect_name

    # If no match, try to extract first meaningful word
    words = re.findall(r'\b[a-zA-Z]{3,}\b', text)
    if words:
        return words[0].title()

    return 'Other'

# Apply the functions to your dataframe
# Assuming your dataframe is called 'df' and you have these columns
df['Part_Name'] = df['DEFECT_PLACE'].apply(extract_main_part)

# You need to replace 'DEFECT_DESCRIPTION' with your actual defect description column name
# Common column names might be: 'DEFECT_TYPE', 'DESCRIPTION', 'DEFECT_DESC', etc.
if 'DEFECT_DESCRIPTION' in df.columns:
    df['Defect_Type'] = df['DEFECT_DESCRIPTION'].apply(extract_defect_type)
elif 'DEFECT_TYPE' in df.columns:
    df['Defect_Type'] = df['DEFECT_TYPE'].apply(extract_defect_type)
else:
    # If you don't have a defect description column, let's use the part name variations
    # or create synthetic categories based on existing data
    print("Available columns:", df.columns.tolist())
    print("Please specify which column contains defect descriptions")
    # For now, create a basic categorization
    df['Defect_Type'] = 'General'

# Create the defect matrix (cross-tabulation)
defect_matrix = pd.crosstab(df['Part_Name'], df['Defect_Type'])

# Filter to show only your specified top 6 parts
specified_parts = ['Door', 'Panel', 'Sidewall', 'Pillar', 'Fender', 'Seat']
# Only include parts that actually exist in your data
top_parts = [part for part in specified_parts if part in defect_matrix.index]
top_defects = df['Defect_Type'].value_counts().head(8).index

# Filter the matrix
defect_matrix_filtered = defect_matrix.loc[top_parts, top_defects]

# Convert to percentages by row (each part sums to 100%)
defect_matrix_pct = defect_matrix_filtered.div(defect_matrix_filtered.sum(axis=1), axis=0) * 100
defect_matrix_pct = defect_matrix_pct.fillna(0)

# Create the attractive heatmap
plt.figure(figsize=(14, 8))

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Create the heatmap with better styling
ax = sns.heatmap(defect_matrix_pct,
                 annot=True,
                 fmt='.1f',
                 cmap='RdYlBu_r',  # Better color scheme
                 linewidths=0.5,   # Add grid lines
                 cbar_kws={'label': 'Percentage (%)'},
                 square=True,      # Square cells
                 annot_kws={'size': 10, 'weight': 'bold'})

# Better title and labels
plt.title('Defect Type Distribution by Component\n(Top 6: Door, Panel, Sidewall, Pillar, Fender, Seat)')
          fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Defect Type', fontsize=12, fontweight='bold')
plt.ylabel('Vehicle Part', fontsize=12, fontweight='bold')

# Rotate labels for better readability
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)

# Adjust layout
plt.tight_layout()

# Add a subtle background color
ax.set_facecolor('#f8f9fa')

plt.show()

# Print some info about the matrix

IndentationError: unexpected indent (ipython-input-2293071466.py, line 145)

In [6]:
# Defect Analysis by Station (Portfolio-Safe Version)

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

# Load a neutral Excel file
file_path = "example_data.xlsx"
df = pd.read_excel(file_path)

print(f"✅ Loaded data from: {file_path}")
print(f"📊 Dataset shape: {df.shape}")
print(f"📋 Columns: {list(df.columns)}")

# Display first few rows to verify data
print(f"\n📋 First 3 rows of data:")
print(df[['STATION', 'DEFECT_TYPE', 'DEFECT_LOCATION']].head(3))

print("🔍 DEFECT CAPTURE ANALYSIS BY STATION")
print("=" * 50)

# Clean station names
df['STATION'] = df['STATION'].str.strip()

# Count defects by station
station_defect_counts = df['STATION'].value_counts()

# Define generic line order
line_order = [
    # Early stages
    'Station A - Quality Gate',
    'Station B - Quality Gate',
    'Station C - Quality Gate',
    'Station D - Quality Gate',

    # Assembly stages
    'Station E - Assembly',
    'Station F - Assembly',
    'Station G - Assembly',
    'Station H - Assembly',
    'Station I - Assembly',
    'Station J - Assembly',

    # Final inspections
    'Station K - Inspection',
    'Station L - Inspection',
    'Station M - Inspection',
    'Station N - Rework',
    'Station O - Audit Complete'
]

# Order stations by line order, then remaining by count
ordered_stations, ordered_counts = [], []

for station in line_order:
    if station in station_defect_counts.index:
        ordered_stations.append(station)
        ordered_counts.append(station_defect_counts[station])

remaining_stations = station_defect_counts.drop(ordered_stations)
for station, count in remaining_stations.items():
    ordered_stations.append(station)
    ordered_counts.append(count)

# Take top 10
top_10_ordered = ordered_stations[:10]
top_10_counts = ordered_counts[:10]

# Print top 10 stations
total_defects = len(df)
print("\n📊 TOP 10 STATIONS BY LINE POSITION (Start → Finish):")
print("-" * 50)
for i, (station, count) in enumerate(zip(top_10_ordered, top_10_counts), 1):
    percentage = (count / total_defects * 100)
    print(f"{i:2d}. {station:<40} | {count:3d} defects ({percentage:.1f}%)")

# Calculate percentages
station_percentages = (station_defect_counts / total_defects * 100).round(2)
print(f"\n📈 PERCENTAGE BREAKDOWN (Total: {total_defects} defects):")
print("-" * 40)
for station, percentage in station_percentages.head(10).items():
    count = station_defect_counts[station]
    print(f"{station:<40} | {percentage:5.1f}% ({count} defects)")

# Visualization
plt.style.use('seaborn-v0_8')
fig = plt.figure(figsize=(20, 14))

# Horizontal bar chart
plt.subplot(2, 1, 1)
bars = plt.barh(range(len(top_10_ordered)), top_10_counts,
                color='steelblue', alpha=0.8, edgecolor='darkblue', linewidth=1)
plt.title('Top 10 Stations by Line Position (Start → Finish)',
          fontsize=18, fontweight='bold', pad=30)
plt.xlabel('Number of Defects', fontsize=14, fontweight='bold')
plt.ylabel('Station', fontsize=14, fontweight='bold')
plt.yticks(range(len(top_10_ordered)), top_10_ordered, fontsize=11)

# Value & percentage labels
for bar, value in zip(bars, top_10_counts):
    plt.text(bar.get_width() + 5, bar.get_y() + bar.get_height()/2,
             f'{value}', ha='left', va='center', fontweight='bold', fontsize=10)
    percentage = (value / total_defects * 100)
    plt.text(bar.get_width()/2, bar.get_y() + bar.get_height()/2,
             f'{percentage:.1f}%', ha='center', va='center',
             color='white', fontweight='bold', fontsize=9)

# Color coding by stage
stage_colors = []
for station in top_10_ordered:
    if 'Quality Gate' in station:
        stage_colors.append('#FF6B6B')  # Early stage
    elif 'Assembly' in station:
        stage_colors.append('#4ECDC4')  # Assembly
    elif 'Inspection' in station or 'Audit' in station or 'Rework' in station:
        stage_colors.append('#45B7D1')  # Final inspection
    else:
        stage_colors.append('#96CEB4')  # Other

for bar, color in zip(bars, stage_colors):
    bar.set_color(color)
    bar.set_alpha(0.8)

plt.grid(axis='x', alpha=0.3)
plt.xlim(0, max(top_10_counts) * 1.15)
plt.gca().invert_yaxis()

# Legend
legend_elements = [
    Patch(facecolor='#FF6B6B', alpha=0.8, label='Early Stage'),
    Patch(facecolor='#4ECDC4', alpha=0.8, label='Assembly'),
    Patch(facecolor='#45B7D1', alpha=0.8, label='Final Inspection'),
    Patch(facecolor='#96CEB4', alpha=0.8, label='Other')
]
plt.legend(handles=legend_elements, loc='lower right', fontsize=10)

# Pie chart
plt.subplot(2, 1, 2)
top_6_stations = station_defect_counts.head(6)
other_count = station_defect_counts[6:].sum()
pie_data = list(top_6_stations.values) + [other_count]

pie_labels = []
for station in top_6_stations.index:
    pie_labels.append(station[:20] + '...' if len(station) > 20 else station)
pie_labels.append('Others')

colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7', '#DDA0DD', '#98D8C8']
wedges, texts, autotexts = plt.pie(pie_data, labels=pie_labels, autopct='%1.1f%%',
                                   startangle=90, colors=colors,
                                   textprops={'fontsize': 10, 'fontweight': 'bold'})
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')

plt.title('Distribution of Defects by Station (Top 6 + Others)',
          fontsize=16, fontweight='bold', pad=20)
plt.legend(wedges, [f"{label} ({count})" for label, count in zip(pie_labels, pie_data)],
           title="Stations", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1), fontsize=9)

plt.tight_layout()
plt.show()

# Additional summary
print(f"\n📋 SUMMARY STATISTICS:")
print("-" * 40)
print(f"Total number of defects: {total_defects}")
print(f"Number of unique stations: {len(station_defect_counts)}")
print(f"Average defects per station: {station_defect_counts.mean():.1f}")
print(f"Station with most defects: {station_defect_counts.index[0]} ({station_defect_counts.iloc[0]} defects)")

# Stations with only 1 defect
single_defect_stations = station_defect_counts[station_defect_counts == 1]
if len(single_defect_stations) > 0:
    print(f"\n⚠️  STATIONS WITH ONLY 1 DEFECT ({len(single_defect_stations)} stations):")
    print("-" * 40)
    for station in single_defect_stations.index[:5]:
        print(f"  • {station}")
    if len(single_defect_stations) > 5:
        print(f"  ... and {len(single_defect_stations) - 5} more")

print(f"\n✅ Analysis complete!")


FileNotFoundError: [Errno 2] No such file or directory: 'example_data.xlsx'

In [7]:
# Calculate percentages for defects by part
part_counts = df['Part_Name'].value_counts()
part_percentages = (part_counts / len(df)) * 100

# Horizontal bar chart for defect percentages by part
plt.figure(figsize=(12, 8))
part_percentages.plot(kind='barh', color='steelblue', edgecolor='darkblue', alpha=0.8)
plt.title('Percentage of Total Defects by Vehicle Part', fontsize=16, fontweight='bold')
plt.xlabel('Percentage of Total Defects (%)', fontsize=12)
plt.ylabel('Vehicle Part', fontsize=12)

# Add percentage labels on bars
for i, (part, pct) in enumerate(part_percentages.items()):
    plt.text(pct + 0.5, i, f'{pct:.1f}%', va='center', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.show()

# Print exact percentages in a clean table
print("📋 Defect Distribution by Vehicle Part:")
print("-" * 40)
for part, pct in part_percentages.items():
    print(f"{part}: {pct:.1f}%")


NameError: name 'df' is not defined

In [8]:
# Clean DEFECT_TYPE column for whitespace issues
df['DEFECT_TYPE'] = df['DEFECT_TYPE'].str.strip()

# Display available defect types
print("Available defect types in data:")
print(df['DEFECT_TYPE'].value_counts())
print("\n" + "="*50 + "\n")

# Categorize defects by type (case-insensitive)
available_defects = df['DEFECT_TYPE'].unique()
dent_defects, air_bubble_defects, scratch_defects = [], [], []

for defect in available_defects:
    defect_lower = defect.lower()
    if 'dent' in defect_lower:
        dent_defects.append(defect)
    elif 'air bubble' in defect_lower:
        air_bubble_defects.append(defect)
    elif 'scratch' in defect_lower:
        scratch_defects.append(defect)

print(f"Dent variations found: {dent_defects}")
print(f"Air Bubble variations found: {air_bubble_defects}")
print(f"Scratch variations found: {scratch_defects}")

# Create filtered DataFrames for each defect category
dent_data = df[df['DEFECT_TYPE'].isin(dent_defects)] if dent_defects else pd.DataFrame()
air_bubble_data = df[df['DEFECT_TYPE'].isin(air_bubble_defects)] if air_bubble_defects else pd.DataFrame()
scratch_data = df[df['DEFECT_TYPE'].isin(scratch_defects)] if scratch_defects else pd.DataFrame()

defect_categories = {
    'Dents': dent_data,
    'Air Bubbles': air_bubble_data,
    'Scratches': scratch_data
}

# Plot top 10 defect locations for each category
fig, axes = plt.subplots(1, 3, figsize=(24, 8))
category_names = list(defect_categories.keys())

for i, (category, defect_data) in enumerate(defect_categories.items()):
    if len(defect_data) > 0:
        # Top 10 locations
        location_counts = defect_data['DEFECT_LOCATION'].value_counts().head(10)
        total_defect = len(defect_data)
        location_percentages = (location_counts / total_defect) * 100

        # Bar chart
        bars = axes[i].bar(range(len(location_counts)), location_counts.values,
                           color=plt.cm.Set3(range(len(location_counts))))
        axes[i].set_title(f'{category} - Top 10 Locations\n(Total: {total_defect})',
                          fontsize=12, fontweight='bold')
        axes[i].set_xlabel('Location')
        axes[i].set_ylabel('Count')
        axes[i].set_xticks(range(len(location_counts)))
        axes[i].set_xticklabels(location_counts.index, rotation=60, ha='right', fontsize=9)

        # Annotate bars with counts and percentages
        for j, (bar, count, pct) in enumerate(zip(bars, location_counts.values, location_percentages.values)):
            axes[i].text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.1,
                         f'{int(count)}\n({pct:.1f}%)',
                         ha='center', va='bottom', fontsize=8, fontweight='bold')
    else:
        axes[i].text(0.5, 0.5, f'No {category} found',
                     ha='center', va='center', transform=axes[i].transAxes,
                     fontsize=12, fontweight='bold')
        axes[i].set_title(f'{category} - Top 10 Locations', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

# Print detailed top 10 breakdown for each defect category
print("TOP 10 LOCATIONS FOR EACH DEFECT CATEGORY")
print("=" * 60)
for category, defect_data in defect_categories.items():
    if len(defect_data) > 0:
        location_counts = defect_data['DEFECT_LOCATION'].value_counts().head(10)
        total_defect = len(defect_data)
        print(f"\n{category.upper()}: Total occurrences: {total_defect}")
        print("-" * 40)
        for location, count in location_counts.items():
            percentage = (count / total_defect) * 100
            print(f"  {location}: {count} ({percentage:.1f}%)")
    else:
        print(f"\n{category.upper()}: No occurrences found")

# Pie charts for top 10 locations per defect category
fig, axes = plt.subplots(1, 3, figsize=(21, 7))

for i, (category, defect_data) in enumerate(defect_categories.items()):
    if len(defect_data) > 0:
        location_counts = defect_data['DEFECT_LOCATION'].value_counts().head(10)
        wedges, texts, autotexts = axes[i].pie(location_counts.values,
                                              labels=location_counts.index,
                                              autopct='%1.1f%%',
                                              startangle=90)
        axes[i].set_title(f'{category} - Top 10 Locations\n(Total: {len(defect_data)})',
                          fontsize=12, fontweight='bold')
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_fontweight('bold')
            autotext.set_fontsize(7)
        for text in texts:
            text.set_fontsize(8)
    else:
        axes[i].text(0.5, 0.5, f'No {category} found',
                     ha='center', va='center', transform=axes[i].transAxes,
                     fontsize=12, fontweight='bold')
        axes[i].set_title(f'{category} - Top 10 Locations', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()


NameError: name 'df' is not defined

In [9]:
# Calculate defect counts and percentages by originator
originator_counts = df['ORIGINATOR'].value_counts()
originator_percentages = (originator_counts / len(df)) * 100

# Select top 5 originators
top5_originator_percentages = originator_percentages.head(5)

# Horizontal bar chart for top 5 originators
plt.figure(figsize=(12, 8))
top5_originator_percentages.plot(kind='barh', color='steelblue', edgecolor='darkblue', alpha=0.8)
plt.title('Percentage of Total Defects by Top 5 Originators', fontsize=14, fontweight='bold')
plt.xlabel('Percentage of Total Defects (%)', fontsize=12, fontweight='bold')
plt.ylabel('Originator Station', fontsize=12, fontweight='bold')

# Annotate bars with percentages
for i, (originator, pct) in enumerate(top5_originator_percentages.items()):
    plt.text(pct + 0.5, i, f'{pct:.1f}%', va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

# Print exact percentages for top 5 originators
print("Defect Distribution by Top 5 Originators:")
print("-" * 40)
for originator, pct in top5_originator_percentages.items():
    print(f"{originator}: {pct:.1f}%")


NameError: name 'df' is not defined